In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import cv2
import os
import time
import shutil
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow

labels = {0: 'bateau', 1: 'bol', 2: 'chat', 3: 'coeur', 4: 'cygne', 5: 'lapin', 6: 'maison', 7: 'marteau', 8: 'montagne', 9: 'pont', 10: 'renard', 11: 'tortue'}

# Must import model.h5 as model and path for the video

video_path = "/content/drive/My Drive/deep/Video.mp4"
model_path = "/content/tangram_jason_mobilenet_final.h5"

model = load_model(model_path)

cam = cv2.VideoCapture(video_path)

# cv2.namedWindow("Camera Shot")

img_counter = 0

if not os.path.exists('frames/'):
    os.makedirs('frames/')
else:
    shutil.rmtree('frames/')
    os.makedirs('frames/')

while True:
    start_time = time.time()
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    print("Camera shot")
    cv2_imshow(frame)

    height, width, dim = frame.shape
    width_cutoff = width // 2
    s1 = frame[:, :width_cutoff]
    s2 = frame[:, width_cutoff:]

    # Resize image to expected size for the model and expansion of dimension from 3 to 4
    s1_up = tf.image.resize(s1, (224,224), preserve_aspect_ratio=False)
    s1_final = tf.expand_dims(s1_up, axis=0)
    s2_up = tf.image.resize(s2, (224,224), preserve_aspect_ratio=False)
    s2_final = tf.expand_dims(s2_up, axis=0)
    
    # Prediction and creation of results dictionnaries
    result_1 = model.predict(s1_final)
    result_dict_1 = {}
    result_2 = model.predict(s2_final)
    result_dict_2 = {}
    for i in range(len(result_1[0])):
        result_dict_1[labels[i]]=result_1[0][i]
        result_dict_2[labels[i]]=result_2[0][i]
    end_time = time.time()
    total_fps = 1/(end_time-start_time)
    print("Total time:",end_time-start_time)
    print("FPS:",total_fps)
    print(result_dict_1)
    print(result_dict_2)

    #Takes a shot every second
    img_name_A = "frames/frame_{}-A.jpg".format(img_counter)
    img_name_B = "frames/frame_{}-B.jpg".format(img_counter)
    cv2.imwrite(img_name_A, s1)
    cv2.imwrite(img_name_B, s2)
    print("{} written!".format(img_name_A.replace("-A.jpg","")))
    
    img_counter += 1
    time.sleep(1)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
        

# cam.release()

# cv2.destroyAllWindows()

# shutil.make_archive('images', 'zip', 'frames/')